In [42]:
pip install galois

In [43]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [44]:
private_seed = secrets.token_bytes(32)
print(private_seed)

b'}\x01H\xb9\xaf\x8aX\xf3-U;#\xbf?]B4;\xfe\xae\xd7\xcc\xbf\x98\xefv)\tT\xb5\x82\xa0'


# SHAKE A.K.A function H (Squeeze public_seed and T)

In [45]:
v = 197
m = 57
lvl = 1

## SHAKE using hashlib

In [46]:
if(lvl==1):
  h_shake = hashlib.shake_128(private_seed)
else:
  h_shake = hashlib.shake_256(private_seed)
private_sponge = h_shake.digest(32+math.ceil(m/8)*v)
print(private_sponge)

b'mLK3p\x8f$bT\xcdv\xc8\x8e\xf2$\x86\xfbt\x96\xe6\x1f\xd1\x93\xd8\xab\xd1H\xfc\x11\x03i\xf0X\x1dG\xff\xdb\x1e\xb6g\xf6\xfd\xfe\x8c\x10\xaf\x83\x88\xbf\xec\x1e\xa2KW\xfd\xddzq\xdd~\xe8d\x82Tti\xb0\x8dR1\xfa\xbd\xe0\x0e,\x8aYsX\x93W\xe3\x8f\xa3\x00&e\x81;\x93\xdfG\xab{\xe1\xd3x\x10\x83L\xe0o\xd9=i\xe2\x0e\x7fg\t\x0b~\x00:Fa\x191\xf7\x80 \xc6W/\x81f5\xe7\xe3\x9d\xad*KH\x94i\x1c\x91\x00\x8f\t\x8d\xfd1\xd38\xa5\x03\x9c\xe7\xef\xda\x89\xa9\x95aD\xafJ\xa4\xb6\x15\xa9yn\x1b\xf493Y\xe1\x9c`\xf3\x82}B\xe7\xd4\xc9\xdb\xf5g\xaf\xac\x89\x83\xac9\x95\xd0\xbcbV\xc0\x94\x1bH\x94\x0cp\xa2?\x7f\x83,\xbd\xb6X-+Dn\xa6\xc1\xfb1\xcd\xf7\xb2\x828\xd1\xff\xfbE\xa6\x8f?^\x7f\xaa\\\x08\xab\xa7\xbe\xfb c3i\x18\xdd\xca\xb7\x0b\xb8\x84\xd7_\xf1\xb2\x99:\xa3\xc1\xda@p\x8c\xfe\x02\x17qK\x16\xf3\xafd\xec}\xda\xe7\xc3\x8d\'\x7f=\xddI\x15\t]\x8d\xa6:E\x94\xfb;\xa1\xd4\xbdeA\x9c\xb0\xba\xef\x1f\x80\x1a~F\xe4\xf6\x92\xc4\tw\xb1)\x85\xa0x.\x02\xd7j}\xf1\xf5\xc6{\x11\xc3\xd8\x0f][Zv\x8f\xe2p\x9c\xc0\x87J1\xbd\xed_\x7f\xc7\

#Extract public seed

In [47]:
public_seed = private_sponge[:32]
print(public_seed)

b'mLK3p\x8f$bT\xcdv\xc8\x8e\xf2$\x86\xfbt\x96\xe6\x1f\xd1\x93\xd8\xab\xd1H\xfc\x11\x03i\xf0'


#Extract T

In [48]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, changed to it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  
  bits_data = bin(intByte)[2:]
  return [int(i) for i in bits_data.zfill(8 * ((len(bits_data) + 7) // 8))]

In [49]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

In [50]:
T_base = private_sponge[32:]
T = tMatrixGenerator(T_base,v,m)
print(T)

[[0 1 0 ... 1 0 1]
 [1 1 1 ... 1 1 0]
 [1 0 1 ... 0 1 1]
 ...
 [0 1 0 ... 1 1 1]
 [1 1 1 ... 1 0 0]
 [0 1 0 ... 1 1 1]]


# SHAKE (squeeze public map)

In [51]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """
  matrix = [int8_to_bits(byteString[i*2])+int8_to_bits(byteString[i*2+1]) for i in range(len(byteString)//2)]
  return np.array(matrix).T

In [52]:
n = m+v
g_equation = 2+2*n+v*(v+1)+2*v*m
mdivided16 = math.ceil(m/16)

#first g function iteration
if(lvl==1):
  g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
else:
  g_shake = hashlib.shake_256(public_seed+bytes.fromhex('00'))
public_sponge = g_shake.digest(g_equation)
C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
public_sponge = public_sponge[2:]
# L and Q1 have more than one dimension, so they are initilized
# differently
L_base = subMatrixGenerator(public_sponge[:2*n])
public_sponge = public_sponge[2*n:]
Q1_base = subMatrixGenerator(public_sponge)

#following g function iterations
if(lvl==1):
  for i in range(mdivided16):
    g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
    public_sponge = g_shake.digest(g_equation)
    C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
    public_sponge = public_sponge[2:]
    L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
    public_sponge = public_sponge[2*n:]
    Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))
else:
  for i in range(mdivided16):
    g_shake = hashlib.shake_256(public_seed+bytes.fromhex(f'0{i}'))
    public_sponge = g_shake.digest(g_equation)
    C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
    public_sponge = public_sponge[2:]
    L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
    public_sponge = public_sponge[2*n:]
    Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))

### Obtaining C

In [53]:
# we take the m bits (starting from the last generated bit) we need from
# the total of bits generated
C = C_base[-m:]
C = np.array(C)
print(C)

[0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0
 1 0 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 0 1]


### Obtaining L

In [54]:
#We discard the upper rows to get the specified dimensions
L = L_base[-m:]
print(L)

[[0 0 0 ... 1 1 0]
 [0 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 1]
 ...
 [0 1 1 ... 0 0 0]
 [0 1 0 ... 0 1 0]
 [1 0 1 ... 1 1 1]]


### Obtaining $$Q_{1}$$

In [55]:
#we discard the upper row to get the specified dimensions
Q1=Q1_base[-m:]
print(Q1)

[[0 1 0 ... 1 1 0]
 [0 1 0 ... 1 0 1]
 [0 1 1 ... 1 0 0]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 1 ... 0 1 0]
 [0 1 0 ... 0 1 0]]


# find $$Q_2$$

In [56]:
def findPk1(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [57]:
def findPk2(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [58]:
def findQ2(Q1:np.ndarray,T:np.ndarray)->galois.GF2:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out of the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(k,Q1))
    Pk2 = GF(findPk2(k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

In [59]:
Q2 = findQ2(Q1,T)
print(Q2)

[[0 0 1 ... 0 1 1]
 [0 1 0 ... 0 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [1 0 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 0 1 ... 1 1 0]]


# Generate the public key

In [60]:
# The first 32 bytes of the public key come from the public seed
public_key = public_seed

# The columns of Q2 are concatenated to get a sequence of bits for the public key
Q2_bits = Q2.flatten('F')
remaining_bits = len(Q2_bits)
Q2_bytes = b''

while(remaining_bits>8):
  byte = ''
  for i in Q2_bits[:8]:
    # Reverse the order of bits
    byte = str(Q2_bits[i]) + byte
  # Turn the bits into a byte and concatenate
  Q2_bytes+=int(byte,2).to_bytes(1,byteorder='big')
  remaining_bits -= 8
  Q2_bits = Q2_bits[8:]
if (remaining_bits>0):
  last_byte = ''
  for bit in Q2_bits:
    # We reverse the order of the
    last_byte = str(bit) + last_byte

  # Add the last byte
  Q2_bytes+=int(last_byte,2).to_bytes(1,byteorder='big')
print(Q2_bytes)

b'\x00~\xff\xff\x00\x00\xff\x00\xff\x00\x8a\xff\xa2\x00*\xaa\xff\xca\x00b\x00\xff\x00\xff\xde\x00\xeeN>\xff\x92\xfa\n\xff\x8a\x96\xe2\x8a\xff\xff\xff\xb2\xba\xfa"j\x00\x00\x00\xff\x00\xb6>\x00\x00\xffv2\x12\xff\x00\xf2\xff\xff\x00F\xff\x82\x1e\xbe*\xff^\x0e\x00\x00\xff\xde\xff\x00\xe6\xff\n\xb6\x00\xee\x1a\xff\xde\x00\x00\xea\x1e\x00\x00\x06\x00\x00\xff\x00f\xae\xff\x0e\xb2\xfeNr\x00\xff\xff\x9e\xff\xff\xff\xff\xfff\xf6\xceB\xff\xaan\x00n\xff\x00\xff\xae\x00\x00\x00\xff\x16\x9a\x00\xe2\xbe\x00f\xff":\x006\xc6Z\x00f\xea\xce\xff\xdaZ\x00\x12FJB\x00\xff\xca\x00&\x00\xff\xff\xe2\x00\x00\x00>\x8eb\xff\x00\x96\x00\xff\x00\xff\xa2\x00r\xff\x1e\xben\x00\x00\x00\x96\xff\xff\xff\x02\x00\xe6\xff\xffN\xf6j\xceJ\xbe\xff\x00\x12\x00\x0e\x00\x00\xff2\xd2f\xae:\x00\x02\xc6\xd2\xff\xe2vj\xff\xff\x16\x1a\x00\xff\xff\xff\xae\xff\x1eVr\x00\xff\x00\xffJ~\xff\x00\x002\xff2\xff\xa6\x06\x00\x00\xba\x00Z&\x16\xff&\x1eb\x82\x00*\x00\x9e\xff\xee\xff\n6\x9e\x9eZ\x00\x06\x00\x00\xba\xf6\x00\x00\xa2:\x00zV\xff\x00\

In [61]:
# Add the bytes to the public key
public_key += Q2_bytes
print(public_key)
print(len(public_key))

b'mLK3p\x8f$bT\xcdv\xc8\x8e\xf2$\x86\xfbt\x96\xe6\x1f\xd1\x93\xd8\xab\xd1H\xfc\x11\x03i\xf0\x00~\xff\xff\x00\x00\xff\x00\xff\x00\x8a\xff\xa2\x00*\xaa\xff\xca\x00b\x00\xff\x00\xff\xde\x00\xeeN>\xff\x92\xfa\n\xff\x8a\x96\xe2\x8a\xff\xff\xff\xb2\xba\xfa"j\x00\x00\x00\xff\x00\xb6>\x00\x00\xffv2\x12\xff\x00\xf2\xff\xff\x00F\xff\x82\x1e\xbe*\xff^\x0e\x00\x00\xff\xde\xff\x00\xe6\xff\n\xb6\x00\xee\x1a\xff\xde\x00\x00\xea\x1e\x00\x00\x06\x00\x00\xff\x00f\xae\xff\x0e\xb2\xfeNr\x00\xff\xff\x9e\xff\xff\xff\xff\xfff\xf6\xceB\xff\xaan\x00n\xff\x00\xff\xae\x00\x00\x00\xff\x16\x9a\x00\xe2\xbe\x00f\xff":\x006\xc6Z\x00f\xea\xce\xff\xdaZ\x00\x12FJB\x00\xff\xca\x00&\x00\xff\xff\xe2\x00\x00\x00>\x8eb\xff\x00\x96\x00\xff\x00\xff\xa2\x00r\xff\x1e\xben\x00\x00\x00\x96\xff\xff\xff\x02\x00\xe6\xff\xffN\xf6j\xceJ\xbe\xff\x00\x12\x00\x0e\x00\x00\xff2\xd2f\xae:\x00\x02\xc6\xd2\xff\xe2vj\xff\xff\x16\x1a\x00\xff\xff\xff\xae\xff\x1eVr\x00\xff\x00\xffJ~\xff\x00\x002\xff2\xff\xa6\x06\x00\x00\xba\x00Z&\x16\xff&\x1eb\x82